In [44]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [45]:
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


#model.fc = torch.nn.Identity()
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)


In [52]:
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os


preprocess=transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class MyDataset(Dataset):


    def __init__(self, csv_file, root_dir, transform=preprocess):

        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        l=self.brix.iloc[idx,5].split(".")
        name=l[0]+"_wb.tiff"
        img_name = os.path.join(self.root_dir,
                                name)


        image = imageio.imread(img_name)
        measures = self.brix.iloc[idx, 1:4]
        measures = np.array([measures])
        avg=np.mean(measures)
        label=np.float32([0])
        if avg>16:
            label=np.float32([1])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image']=np.float32(sample['image'])
            sample['image'] = self.transform(sample['image'])

        return sample

brix_dataset = MyDataset(csv_file='/datalab/brix_labels_07_ott_2021-Foglio1.csv',
                                    root_dir='/datalab')

input_image=brix_dataset[2]['image']
print(brix_dataset[2]['label'])
input_tensor=input_image



input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

[1.]
tensor([1.], device='cuda:0')


In [53]:
n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
dataset_size=MyDataset.__len__(brix_dataset)

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for batch_idx in range(dataset_size):
     sample=brix_dataset[batch_idx]
     data_=sample['image']
     target_=torch.tensor(sample['label']).unsqueeze(0)
     if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
     optimizer.zero_grad()
     data_ = data_.unsqueeze(0)      
     outputs = model(data_)
     probabilities = torch.nn.functional.softmax(outputs)
     #print(outputs,"and",probabilities)
     loss = criterion(probabilities, target_)
     loss.backward()
     optimizer.step()
     running_loss += loss.item()
     #_,pred = torch.max(outputs, dim=1)
     if probabilities[0]>0.5:
       pred=torch.tensor(np.float32([1]))
     else:
        pred=torch.tensor(np.float32([0]))
     pred=pred.to('cuda')
     correct += torch.sum(pred==target_).item()
     total += target_.size(0)
     if (batch_idx) % 20 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        .format(epoch, n_epochs, batch_idx, dataset_size, loss.item()))
  train_acc.append(100 * correct / total)
  train_loss.append(running_loss/dataset_size)
  print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
  batch_loss = 0  
  total_t=0
  correct_t=0
  #VALIDATE
  #with torch.no_grad():
    #model.eval()
    #for data_t, target_t in (test_dataloader):
      #if torch.cuda.is_available():
        #data_t, target_t = data_t.to('cuda0'), target_t.to('cuda0')
      #outputs_t = model(data_t)
      #loss_t = criterion(outputs_t, target_t)
      #batch_loss += loss_t.item()
      #_,pred_t = torch.max(outputs_t, dim=1)
      #correct_t += torch.sum(pred_t==target_t).item()
      #total_t += target_t.size(0)
      #val_acc.append(100 * correct_t/total_t)
      #val_loss.append(batch_loss/len(test_dataloader))
      #network_learned = batch_loss < valid_loss_min
      #print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')     
      #if network_learned:
        #valid_loss_min = batch_loss
        #torch.save(net.state_dict(), 'resnet.pt')
        #print('Improvement-Detected, save-model')
      #model.train()
  


Epoch 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/5], Step [0/50], Loss: -0.0000
Epoch [1/5], Step [20/50], Loss: -0.0000
Epoch [1/5], Step [40/50], Loss: -0.0000

train-loss: 0.0000, train-acc: 100.0000
Epoch 2

Epoch [2/5], Step [0/50], Loss: -0.0000
Epoch [2/5], Step [20/50], Loss: -0.0000


KeyboardInterrupt: ignored

In [3]:
torch.cuda.is_available()

True